# Laborator 8

## Modele de regresie

Folositi urmatoarele seturi de date:
1. [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".
1. [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)
1. [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"
1. [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

Pentru fiecare set de date aplicati minim 5 modele de regresie din scikit learn. Pentru fiecare raportati: mean absolute error, mean squared error, median absolute error - a se vedea [sklearn.metrics](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) - folosind 5 fold cross validation. Valorile hiperparametrilor trebuie cautate cu grid search (cv=3)  si random search (n_iter dat de voi). Metrica folosita pentru cautarea hiperparametrilor va fi mean squared error. Raportati mediile rezultatelor atat pentru fold-urile de antrenare, cat si pentru cele de testare; indicatie: puteti folosi metoda `cross_validate` cu parametrul `return_train_score=True`, iar ca model un obiect de tip `GridSearchCV` sau `RandomizedSearchCV`.

Rezultatele vor fi trecute intr-un dataframe. Intr-o stare intermediara, valorile vor fi calculate cu semnul minus: din motive de implementare, biblioteca sklearn transforma scorurile in numere negative; a se vedea imaginea de mai jos:

![intermediate report](./images/cpu_intermediate_blurred.png)


Valorile vor fi aduse la interval pozitiv, apoi vor fi marcate cele maxime si minime; orientativ, se poate folosi imaginea de mai jos, reprezentand dataframe afisat in notebook; puteti folosi alte variante de styling pe dataframe precum la https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#.  

Se va crea un raport final in format HTML sau PDF - fisier(e) separat(e). Raportul trebuie sa contina minimal: numele setului de date si obiectul dataframe; preferabil sa se pastreze marcajul de culori realizat in notebook.

![report](./images/cpu_results_blurred.png)

Notare:
1. Se acorda 20 de puncte din oficiu.
1. Optimizare si cuantificare de performanta a modelelor: 3 puncte pentru fiecare combinatie set de date + model = 60 de puncte
1. Documentare modele: numar modele * 2 puncte = 10 puncte. Documentati in jupyter notebook fiecare din modelele folosite, in limba romana. Puteti face o sectiune separata cu documentarea algoritmilor. Fiecare model trebuie sa aiba o descriere de minim 20 de randuri, minim o imagine asociata si minim 2 referinte bibliografice.
1. 10 puncte: export in format HTML sau PDF.



*Notare:* laboratorul va fi salvat in repository-ul de github si prezentat in saptamana 6-10 mai.

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

print('Pandas version:', pd.__version__) #Pandas version: 0.23.4

Pandas version: 0.24.1


# Loading and preparing the data sets.

## [CPU Computer Hardware](https://archive.ics.uci.edu/ml/datasets/Computer+Hardware); excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance"

In [2]:
dataframe_machine = pd.read_csv("data/machine.data", header = None)

In [3]:
dataframe_machine.head()

,0,1,2,3,4,5,6,7,8,9
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


In [4]:
dataframe_machine = dataframe_machine.drop([0, 1, 9], axis = 1)
dataframe_machine.head()

,2,3,4,5,6,7,8
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [5]:
dataframe_machine_X = dataframe_machine.values[:, :-1]
dataframe_machine_Y = dataframe_machine.values[:, -1]

## [Boston Housing](http://archive.ics.uci.edu/ml/machine-learning-databases/housing/)

In [6]:
dataframe_housing = pd.read_csv("data/housing.data", header = None, sep=r"\s+")

In [7]:
dataframe_housing.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [8]:
dataframe_housing_X = dataframe_housing.values[:, :-1]
dataframe_housing_Y = dataframe_housing.values[:, -1]

## [Wisconsin Breast Cancer](http://www.dcc.fc.up.pt/~ltorgo/Regression/DataSets.html); cautati in panelul din stanga Wisconsin Breast Cancer si urmati pasii din "My personal Notes"

In [9]:
dataframe_wpbc = pd.read_csv("data/r_wpbc.data", header = None)

In [10]:
dataframe_wpbc.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,5,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,31
1,2,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,61
2,0,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,116
3,0,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,123
4,0,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,27


In [11]:
dataframe_wpbc_X = dataframe_wpbc.values[:, :-1]
dataframe_wpbc_Y = dataframe_wpbc.values[:, -1]

## [Communities and Crime](http://archive.ics.uci.edu/ml/datasets/communities+and+crime); stergeti primele 5 dimensiuni si trasaturile cu missing values.

In [12]:
dataframe_communities = pd.read_csv("data/communities.data", header = None)

In [13]:
dataframe_communities.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


In [14]:
dataframe_communities = dataframe_communities.drop([0,1,2,3,4], axis=1)
missing_index = dataframe_communities.columns[(dataframe_communities == '?').any()].tolist()
dataframe_communities = dataframe_communities.drop(missing_index, axis=1)
dataframe_communities.head()

C:\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,5,6,7,8,9,10,11,12,13,14,...,96,97,98,99,100,118,119,120,125,127
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03


In [15]:
dataframe_communities_X = dataframe_communities.values[:, :-1]
dataframe_communities_Y = dataframe_communities.values[:, -1]

# Defining the regression models and auxiliary functions.

In [16]:
data = []
columns = ['Model', 'Search', 'train_mean_absolute_error', 'train_mean_squared_error',
            'train_median_absolute_error', 'test_mean_absolute_error', 'test_mean_squared_error', 
            'test_median_absolute_error']
subset = ['train_mean_absolute_error', 'train_mean_squared_error', 'train_median_absolute_error',
            'test_mean_absolute_error', 'test_mean_squared_error', 'test_median_absolute_error']

## Highlighting functions for pandas data frame

In [17]:
def highlight_max(s)-> pd.DataFrame:
    '''
    Colors the max cells in green, given a pandas DataFrame object.
    param:  s: the pandas DataFrame
    return: colored pandas DataFrame
    '''
    is_max = s == s.max()
    return ['background-color: red' if v else '' for v in is_max]

In [18]:
def highlight_min(s)-> pd.DataFrame:
    '''
    Colors the min cells in red, given a pandas DataFrame object.
    param:  s: the pandas DataFrame;
    return: colored pandas DataFrame
    '''
    is_min = s == s.min()
    return ['background-color: green' if v else '' for v in is_min]

## Errors function, mean square, mean absolute, median.

In [20]:
def errors(model, x_train :np.ndarray, x_test :np.ndarray, y_train :np.ndarray, y_test: np.ndarray) -> list:
    '''
    Calculates the 'neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error' on both train and
    test sets with cross validation, having the model as a param.
    param:   model: sklearn model, the given model that is needed for cross_validation;
             x_train: the train set for the model;
             x_test: the test set for the model;
             y_train: the verification for the train set;
             y_test: the verification for the test set;
    return:  list with the errors multyplied by -1 to be positive.
    '''
    train_neg_mean_absolute_error = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_mean_absolute_error')
    train_neg_mean_squared_error = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_mean_squared_error')
    train_neg_median_absolute_error = cross_val_score(model, x_train, y_train, cv=5, scoring='neg_median_absolute_error')
    test_neg_mean_absolute_error = cross_val_score(model, x_test, y_test, cv=5, scoring='neg_mean_absolute_error')
    test_neg_mean_squared_error = cross_val_score(model, x_test, y_test, cv=5, scoring='neg_mean_squared_error')
    test_neg_median_absolute_error = cross_val_score(model, x_test, y_test, cv=5, scoring='neg_median_absolute_error')
    return [train_neg_mean_absolute_error.mean()*-1, train_neg_mean_squared_error.mean()*-1, 
           train_neg_median_absolute_error.mean()*-1, test_neg_mean_absolute_error.mean()*-1, 
           test_neg_mean_squared_error.mean()*-1, test_neg_median_absolute_error.mean()*-1]

## Grid search and random search functions.

In [22]:
def search_random(parameters: dict, model, x_train: np.ndarray, x_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> list:
    '''
    Dose the random cross validation search, it returns the best parameter found and than, the model goes to the error
    function to see how well it has done.
    param:   parameters: dictionary type, has the all the alternatives to the hyperparameters, the actual subject of
                         this function
             model: sklearn model, the given model that is needed for cross_validation;
             x_train: the train set for the model;
             x_test: the test set for the model;
             y_train: the verification for the train set;
             y_test: the verification for the test set;
    return:  list with erros returned by 'errors' function and the appropiate name.
    '''
    randomized_search = RandomizedSearchCV(estimator = model, param_distributions = parameters, scoring='neg_mean_squared_error', cv=3, return_train_score=True, iid=False)
    randomized_search.fit(x_train, y_train)
    return ['Random Search'] + errors(randomized_search, x_train, x_test, y_train, y_test)

In [24]:
def search_grid(parameters: dict, model, x_train: np.ndarray, x_test: np.ndarray, y_train: np.ndarray, y_test: np.ndarray) -> list:
    '''
    Dose the grid cross validation search, it returns the best parameter found and than, the model goes to the error
    function to see how well it has done.
    param:   parameters: dictionary type, has the all the alternatives to the hyperparameters, the actual subject of
                         this function
             model: sklearn model, the given model that is needed for cross_validation;
             x_train: the train set for the model;
             x_test: the test set for the model;
             y_train: the verification for the train set;
             y_test: the verification for the test set;
    return:  list with erros returned by 'errors' function and the appropiate name.
    '''
    grid_search = GridSearchCV(estimator = model, param_grid = parameters, scoring='neg_mean_squared_error', cv=3, return_train_score=True, iid=False)
    grid_search.fit(x_train, y_train)
    return ['Grid Search'] + errors(grid_search, x_train, x_test, y_train, y_test)

## Model functions, KNN, SVR, Random Forest, Tree, GaussianProcessRegressor.

### KNeighborsRegressor

In [25]:
def search_knn(parameters: dict, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray) -> None:
    '''
    Initialization for KNeighborsRegressor model and starting the grid and random search.
    Appends into a list the results
    '''
    model = KNeighborsRegressor()
    data.append(['KNN'] + search_grid(parameters, model, x_train, x_test, y_train, y_test))
    data.append(['KNN'] + search_random(parameters, model, x_train, x_test, y_train, y_test))

### SVR

In [26]:
def search_svc(parameters: dict, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray) -> None:
    '''
    Initialization for SVR model and starting the grid and random search.
    Appends into a list the results
    '''
    model = SVR()
    data.append(['SVR'] + search_grid(parameters, model, x_train, x_test, y_train, y_test))
    data.append(['SVR'] + search_random(parameters, model, x_train, x_test, y_train, y_test))

### RandomForestRegressor

In [27]:
def search_forest(parameters: dict, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray) -> None:
    '''
    Initialization for RandomForestRegressor model and starting the grid and random search.
    Appends into a list the results
    '''
    model = RandomForestRegressor()
    data.append(['Forest'] + search_grid(parameters, model, x_train, x_test, y_train, y_test))
    data.append(['Forest'] + search_random(parameters, model, x_train, x_test, y_train, y_test))

### DecisionTreeRegressor

In [28]:
def search_tree(parameters: dict, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray) -> None:
    '''
    Initialization for DecisionTreeRegressor model and starting the grid and random search.
    Appends into a list the results
    '''
    model = DecisionTreeRegressor()
    data.append(['Tree'] + search_grid(parameters, model, x_train, x_test, y_train, y_test))
    data.append(['Tree'] + search_random(parameters, model, x_train, x_test, y_train, y_test))

### GaussianProcessRegressor

In [29]:
def search_GaussianProcessRegressor(parameters: dict, x_train: np.ndarray, y_train: np.ndarray, x_test: np.ndarray, y_test: np.ndarray) -> None:
    '''
    Initialization for GaussianProcessRegressor model and starting the grid and random search.
    Appends into a list the results
    '''
    model = GaussianProcessRegressor()
    data.append(['GaussianProcessRegressor'] + (search_grid(parameters, model, x_train, x_test, y_train, y_test)))
    data.append(['GaussianProcessRegressor'] + (search_random(parameters, model, x_train, x_test, y_train, y_test)))

# The main function to deliver the search.

In [30]:
def search(x: np.ndarray, y: np.ndarray, name: str) -> None:
    '''
    Creates a pandas DataFrame object in witch it stores all the results that are needed. Exports the dataframe into a 
    HTML file.
    param:   x: the data set for the model;
             y: the labels set for the model;
             name: the name of export HTML file
    return:  None
    '''
    data.clear()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1/3)
    
    parameters_gaussian = {'alpha': list([0.0000000000001, 0.000000000001, 0.00000000001, 0.0000000001, 0.000000001, 0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0])}
    parameters_tree = {'random_state': list([1, 50, 100, 200, 300, 333, 500, 700, 1000, 2000, 3000, 10000])}
    parameters_forest = {'n_estimators': list(range(1,13))}
    parameters_svr = {'gamma' : list(['auto']), 'C': list([0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 2.0, 3.0, 4.0, 5.0, 10.0])}
    parameters_knn = {'n_neighbors': list(range(1,12)), 'p': [1, 2, 3, 4.7]} 
    
    search_knn(parameters_knn, x_train, y_train, x_test, y_test)
    search_svc(parameters_svr, x_train, y_train, x_test, y_test)
    search_forest(parameters_forest, x_train, y_train, x_test, y_test)
    search_tree(parameters_tree, x_train, y_train, x_test, y_test)
    search_GaussianProcessRegressor(parameters_gaussian, x_train, y_train, x_test, y_test)
    
    data_frame = pd.DataFrame(data, columns=columns)
    data_frame = data_frame.style.apply(highlight_max, subset=subset).apply(highlight_min, subset=subset)
    f=open(f"{name}.html","w")
    f.write(data_frame.render())
    f.close()

# Feeding the models with data

## Housing

In [31]:
search(dataframe_housing_X, dataframe_housing_Y, "housing")

## Machine

In [32]:
search(dataframe_machine_X, dataframe_machine_Y, "machine")

## Wpbc

In [33]:
search(dataframe_wpbc_X, dataframe_wpbc_Y, "wpbc")

## Communities

In [ ]:
search(dataframe_communities_X, dataframe_communities_Y, "communities")

# Documentation

## 1. K-Nearest Neighbors Regression (KNN)

K-Nearest Neighbors Regression este un algoritm de invatare nesupervizata, conceput pentru a fi utilizat în probleme care implică segmentarea pieței, detectarea fraudei, motoare de recomandare, gruparea paginilor web prin similitudine.

În KNN, K reprezinta numărul de vecini apropiați. Numărul de vecini este factorul decisiv de bază. K este, în general, un număr impar dacă numărul de clase este 2. Când K = 1 algoritmul este cunoscut ca nearest neighbor. Acesta este cel mai simplu caz. Sa presupunem că P1 este un punct pentru care trebuie sa ii prezicem eticheta. Mai întâi, găsim cel mai apropiat punct de P1 și atribuim eticheta acestuia lui P1.

![report](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/Knn_k1_z96jba.png)

Cand K > 1 algoritmul cauta K cei mai apropiati vecini de punctul P1 si eticheta lui este bazata pe votul majoritatii vecinilor. Pentru a gasi cele mai apropiate puncte similare de P1, trebuie calculate distantele dintre puncte folosind metrici precum distanta Euclidiana, distanta Hamming, distanta Manhattan si distanta Minkowski. KNN are urmatoarele etape de baza:

1. Calculculeaza distanta
2. Gaseste cel mai apropiat vecin
3. Voteaza pentru eticheta

![report](http://res.cloudinary.com/dyd911kmh/image/upload/f_auto,q_auto:best/v1531424125/KNN_final1_ibdm8a.png)

Nu este nevoie de învățare sau instruire a modelului și a tuturor punctelor de date folosite la momentul predicției. Algoritmul  așteaptă până în ultimul minut pentru a clasifica orice punct. Acesta stochează doar setul de date de antrenament și așteaptă până când clasificarea trebuie să se desfășoare. Doar atunci când vede setul de testare, efectuează generalizarea pentru a clasifica setul de testare bazat pe similitudinea cu setul de antrenament stocat. Spre deosebire de alte metodele de învățare, algoritmul face mai puțini pasi în faza de formare și mai multă muncă în faza de testare pentru a face o clasificare.

K-nearest neighbors regression folosește ponderi uniforme, adică fiecare punct al unei clase contribuie uniform la identificarea unui punct de interogare. În anumite circumstanțe, poate fi avantajos să se analizeze puntele astfel încât punctele din apropiere să contribuie mai mult la regresie decât punctele îndepărtate. Acest lucru poate fi realizat prin intermediul cuvântului cheie ponderi. Valoarea implicită, weights = "uniform", atribuie ponderi egale tuturor punctelor. weights = 'distance' atribuie ponderi proporționale cu inversul distanței de la punctul de interogare. Alternativ, poate fi furnizată o funcție definită de utilizator a distanței, care va fi utilizată pentru a calcula ponderile.

Cercetările au arătat că nici un număr optim de vecini nu se potrivește cu toate tipurile de seturi de date. Fiecare set de date are propriile cerințe. În cazul unui număr mic de vecini, zgomotul va avea o influență mai mare asupra rezultatului, și un număr mare de vecini îl face costisitor din punct de vedere computațional. Cercetările au arătat, de asemenea, că o cantitate mică de vecini este cea mai flexibilă. În general, se alege K un număr impar dacă numărul de clase este par.


Resurse:
1. https://www.datacamp.com/community/tutorials/k-nearest-neighbor-classification-scikit-learn
2. https://analyticstraining.com/popular-regression-algorithms-ml/
3. https://scikit-learn.org/stable/modules/neighbors.html#regression


## 2. Support Vector Regression (SVR)

Support Vector Regression este utilizat în machine learning pentru a rezolva problemele legate de segmentarea imaginii, piața bursieră, clasificarea textului și științele biologice.

Support Vector Machines este o clasa foarte specifica de algoritmi, caracterizata prin folosirea kernelurilor, absenta minimelor locale, dispersie a solutiei si controlul capacitatii obtinute prin actiunea asupra marginilor sau pe numarul de support vectors etc.

SVR folosește aceleași principii ca SVM pentru clasificare, cu doar câteva diferențe minore. În primul rând, deoarece output-ul este un număr real, devine foarte dificilă prezicerea informațiilor care au posibilități infinite. În cazul regresiei, o marjă de toleranță (epsilon) este stabilită în apropierea SVM care ar fi solicitat deja din problemă. Cu toate acestea, ideea principală este întotdeauna aceeași: pentru a minimiza eroarea se individualizeaza hiperplanul care maximizează marja, având în vedere că o parte din eroare este tolerată. 

*Kernel-ul este funcția utilizată pentru a mapa date dimensionale inferioare într-o valoare dimensională superioară.

*Hyper Plane in SVR este definit ca linia care ne va ajuta să anticipăm valoarea continuă sau valoarea țintă.

*Support Vectors sunt punctele care se află cel mai aproape de limită. Distanța punctelor este minimă.

*Boundary line: în SVM există alte două linii decât Hyper Plane care creează o marjă. Support Vectors pot fi pe Boundary line sau în afara ei. Această linie de separare separă cele două clase. În SVR, conceptul este același.

Exista 2 tipuri de SVR:
1. Linear
2. Non-Linear

![raport](https://www.saedsayad.com/images/SVR_5.png)

A se vedea modul în care toate punctele se află între boundaryes lines. Obiectivul nostru este ca atunci când ne mișcăm cu SVR sa luam în considerare punctele care se află între boundaryes lines. Linia de potrivire cea mai bună este linia hyperplane care are un număr maxim de puncte.

![](https://cdn-images-1.medium.com/freeze/1*rs0EfF8RPVpgA-EfgAq85g.jpeg)

Modelul produs de Support Vector Regression depinde numai de un subset de date de antrenament, deoarece funcția de cost pentru construirea modelului ignoră toate datele de antrenament apropiate de predicția modelului.

Resurse:
1. https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff
2. https://analyticstraining.com/popular-regression-algorithms-ml/
3. https://www.saedsayad.com/support_vector_machine_reg.htm
4. https://scikit-learn.org/stable/modules/svm.html#svm-regression

## 3. Random Forest Regressor

Random Forest Regressor este utilizat într-o mulțime de domenii diferite, cum ar fi sectorul bancar, bursa, medicina și comerțul electronic. În sectorul bancar se utilizează, de exemplu, pentru a detecta clienții care vor folosi serviciile băncii mai frecvent decât alții și își vor rambursa datoriile în timp. În acest domeniu, este folosit și pentru detectarea clienților de fraudă care doresc să înșele banca. În finanțe, este folosit pentru a determina comportamentul stocului în viitor. În domeniul sănătății este folosit pentru a identifica combinația corectă a componentelor din medicină și pentru a analiza istoricul medical al pacientului pentru a identifica bolile. Și, în sfârșit, in comerțul electronic este utilizat pentru a determina dacă un client va dori de fapt produsul sau nu.

Random forest este o tehnică de ansamblu capabilă să efectueze atât sarcini de regresie, cât și sarcini de clasificare, folosind mai mulți copaci de decizie și o tehnică numită Bootstrap Aggregation, cunoscută în mod obișnuit ca "bagging". Bagging-ul, în metoda Random Forest, implică instruirea fiecărui arbore al deciziilor pe un eșantion de date diferit, unde prelevarea de probe se face cu înlocuirea.

Ideea de bază din spatele acestui algoritm este combinarea mai multor arbori de decizie în determinarea output-ului final, în loc să se bazeze pe arborii decizionali individuali.

![](https://cdn-images-1.medium.com/max/1200/1*i0o8mjFfCn-uD79-F1Cqkw.png)

Random forest adaugă o randomizare suplimentară modelului, în timp ce crește copacii. În loc să căutăm cea mai importantă caracteristică în timp ce divizăm un nod, acesta caută cea mai bună caracteristică dintr-un subset aleatoriu de caracteristici. Aceasta are ca rezultat o diversitate largă care, în general, are ca rezultat un model mai bun.

Prin urmare, în random forest, doar un subset aleator al caracteristicilor este luat în considerare prin algoritmul de împărțire a unui nod. Se pot face arbori mai aleatorii, folosind în plus praguri aleatorii pentru fiecare caracteristică, în loc să căutați cele mai bune praguri posibile (cum face un decision tree).

![](https://cdn-images-1.medium.com/max/1600/1*9kACduxnce_JdTrftM_bsA.gif)

Resuse:
1. https://medium.com/datadriveninvestor/random-forest-regression-9871bc9a25eb
2. https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd
3. https://medium.com/@williamkoehrsen/random-forest-simple-explanation-377895a60d2d

## 4. Decision Tree Regressor

Decision Tree este un model intuitiv în care se traversează cate o data ramurile copacului și selectează următoarea ramură pentru a merge în jos pe baza unei decizii la un nod. Inducția copacilor este sarcina de a lua un set de instanțe de formare ca intrări, de a decide care atribute sunt cel mai bine să se împartă, de a împărți setul de date și de a repeta pe seturile de date divizate rezultate până când toate instanțele de formare sunt clasificate. 

În timpul construirii copacului, scopul este de a se împărți pe atributele care creează cele mai pure noduri copil posibile, ceea ce ar reduce la minimum numărul de împărțiri care ar trebui făcute pentru a clasifica toate instanțele din setul nostru de date. Puritatea este măsurată prin conceptul de câștig de informații, care se referă la cât de mult ar trebui să fie cunoscuta o instanță anterior nevăzută pentru a fi clasificată corespunzător. 

![](https://cdn-images-1.medium.com/max/800/1*KOg1s4s4mg44q_6Oi5D-mQ.png)

În practică, acest lucru se măsoară prin compararea entropiei sau a cantității de informații necesare pentru a clasifica o singură instanță a unei partiții de date curente la cantitatea de informații pentru a clasifica o singură instanță în cazul în care divizia de seturi de date curente urma să fie mai mult împărțită pe un anumit atribut.

![](https://cdn-images-1.medium.com/max/600/1*XMId5sJqPtm8-RIwVVz2tg.png)

O problemă des intalnita este faptul ca un set mare de caracteristici, rezultă un număr mare de împărțire, care la rândul său conduce la un copac uriaș. Asemenea copaci sunt complexi și pot duce la suprasolicitare. Deci, trebuie să știm când să ne oprim. O modalitate de a face acest lucru este de a stabili un număr minim de intrări de instruire pe fiecare frunză. De exemplu, putem folosi cel puțin 10 pasageri pentru a ajunge la o decizie (a murit sau a supraviețuit) și pentru a ignora orice frunză care ia mai puțin de 10 pasageri. O altă modalitate este de a stabili adâncimea maximă a modelului. Adâncimea maximă se referă la lungimea celei mai lungi căi de la o rădăcină la o frunză.

Resurse:
1. https://towardsdatascience.com/selecting-the-best-machine-learning-algorithm-for-your-regression-problem-20c330bad4ef
2. https://towardsdatascience.com/decision-trees-in-machine-learning-641b9c4e8052
3. https://scikit-learn.org/stable/modules/tree.html#tree

## 5. Gaussian Process Regression (GPR)

Gaussian Process este un algoritm puternic atât pentru regresie, cât și pentru clasificare. Cel mai mare avantaj practic al acestuia este că poate oferi o estimare fiabilă a propriei incertitudini. 

Gaussian Process Regressor implementează Gaussian Process (GP) în scopuri de regresie. Pentru aceasta, trebuie precizat înainte GP. Media anterioară se presupune a fi constantă și zero (pentru normalize_y = False) sau media datelor de antrenament (pentru normalize_y = True). Co-covarianța precedentului este specificată prin trecerea unui obiect kernel. Hiperparametrele kernel-ului sunt optimizate în timpul montajului GPR prin maximizarea probabilității log-marginale (LML) bazată pe optimizatorul trecut. Deoarece LML poate avea mai multe optimizări locale, optimizatorul poate fi pornit în mod repetat prin specificarea n_restarts_optimizer. Prima rulare se efectuează întotdeauna pornind de la valorile inițiale ale hiperparametrului kernel-ului; etapele următoare sunt efectuate din valorile hiperparametrului care au fost alese aleatoriu din intervalul de valori permise. Dacă hiperparametrele inițiale ar trebui să fie ținute fixe, niciuna nu poate fi trecută ca optimizator.

Nivelul de zgomot poate fi specificat prin trecerea acestuia prin parametrul alfa, fie global ca scalar, fie per datapoint. Rețineți că un nivel moderat al zgomotului poate fi, de asemenea, util pentru rezolvarea problemelor numerice în timpul montajului, deoarece este efectiv implementat ca regularizare Tikhonov, adică prin adăugarea acestuia la diagonala matricei kernel-ului. O alternativă la specificarea nivelului de zgomot în mod explicit este includerea unei componente WhiteKernel în kernel, care poate estima nivelul global de zgomot din date.

Acest exemplu ilustrează faptul că GPR cu un kernel sum care include un WhiteKernel poate estima nivelul de zgomot al datelor. O ilustrare a peisajului logargin-probabilității (LML) arată că există două maxime locale ale LML.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_gpr_noisy_0001.png)

Primul corespunde unui model cu un nivel ridicat al zgomotului și o scală de lungime mare, care explică toate variațiile datelor prin zgomot.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_gpr_noisy_0011.png)

Cel de-al doilea are un nivel de zgomot mai mic și o scală de lungime mai scurtă, ceea ce explică cea mai mare parte a variației prin relația funcțională fără zgomot. Cel de-al doilea model are o probabilitate mai mare; totuși, în funcție de valoarea inițială pentru hiperparametrii, optimizarea pe bază de gradient ar putea converge, de asemenea, la soluția cu zgomot ridicat. Este deci important să repetați optimizarea de mai multe ori pentru diferite inițializări.

Resurse:
1. https://scikit-learn.org/stable/modules/gaussian_process.html
2. https://towardsdatascience.com/an-intuitive-guide-to-gaussian-processes-ec2f0b45c71d